# Parsing APIs Example

## Intro

Now we will take a look on a real data. When you parse data from web you will often meet API based web-pages. 

For example [zalando.fr](https://www.zalando.fr/accueil-homme/) is API based web-page. 

In this guided lab you will learn how to obtain the links from webpages and extract the data. Read through this doc, execute the cells in order and make sure you understand the explanations. 

*Note: This guided lab uses Google Chrome. Other browsers like Safari and Firefox have similar tools for developers but they work differently. To save your time in following this lab, it is strongly recommended that you install and use Google Chrome.*

## Obtaining the link

Zalando is discount e-store where you can buy clothes and accesories with discount. When we go to the web-page, we can choose different sections. First the general process will be shown using [Children section](https://www.zalando.fr/accueil-enfant/) as example.

Here we will parse data about promotions only. Therefore, final output will be the DataFrame with all the goods under discount.

[![Image from Gyazo](https://i.gyazo.com/fa4874d8e81c7570273bbfb853d66308.png)](https://gyazo.com/fa4874d8e81c7570273bbfb853d66308)


We go to Promos page. Right click of mouse shows us a list of actions possible, from which we select Inspect.

<img src='https://i.gyazo.com/bccbd11d69c9040dc98758d443e32052.png' width="400">


You will see the menu dropdown on the right side or on the bottom of the window. There you should click on Network:


[![Image from Gyazo](https://i.gyazo.com/f7e0db81cbfee67694183d1a7640bf81.png)](https://gyazo.com/f7e0db81cbfee67694183d1a7640bf81)

Right after the developer part will change showing the files behind the page. In order to obtain only useful files we select the following settings:
1. Preserve Log
2. Select XHR files.

[![Image from Gyazo](https://i.gyazo.com/9a899d4441d9d93e795f79747f1e47d5.png)](https://gyazo.com/9a899d4441d9d93e795f79747f1e47d5)

In order to obtain some files we need to scrool down and go forward to second page. 

[![Image from Gyazo](https://i.gyazo.com/0956eb3d5125075a236c9a439c7749c7.png)](https://gyazo.com/0956eb3d5125075a236c9a439c7749c7)

In the Network panel you can see the following files being uploaded. All the data on the web-page is uploaded from the json file, which is one of the following. It is important to understand which file contains what kind of information. 

<a href="https://gyazo.com/cf97a655869f0b22df0ada1cb2a41c3c"><img src="https://i.gyazo.com/cf97a655869f0b22df0ada1cb2a41c3c.png" alt="Image from Gyazo" width="724.8"/></a>

When you find what kind of information you need for the data to be uploaded you just test it. Here we need the article... file:

<a href="https://gyazo.com/78b35bf492994b3f35c0564a21da202a"><img src="https://i.gyazo.com/78b35bf492994b3f35c0564a21da202a.png" alt="Image from Gyazo" width="727.2"/></a>

When we test the link in Chrome inkognito mode we obtain the proper json file:


<a href="https://gyazo.com/b60453fa98454fa29771c731a5174443"><img src="https://i.gyazo.com/b60453fa98454fa29771c731a5174443.png" alt="Image from Gyazo" width="1530.4"/></a>

In order to change the objects in the json file (kind of pagination), you need to change the offset (the number of the first element on the page). in fact, if you take a look on the link, it is easy to unerstand the structure of the link.

# Reading the data

Now the party rocks! When we know how can we obtain the data, it is not a problem to obtain the whole database with all the data from the web-page.
In this lab you will collect your database of Zalando products. You select which goods you want to track. You can define as many filters to your data as you want. Just make sure that the data represents the filters.




In [1]:
import json
import requests
import pandas as pd
from pandas.io.json import json_normalize


In [2]:
# Paste the url you obtained for your data
url='https://www.zalando.fr/api/catalog/articles?categories=promo-enfant&limit=84&offset=84&sort=sale'


#### Collect first 84 object of the of the data (1st page)

Your output should be a Pandas DataFrame of goods. Each row should contain only text or numbers, having *family_articles, flags, media* and *sizes* remaining lists (they are exceptions). Hint: use the headers parameter to get the data!

In [3]:
# headers definition

headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_2) Chrome/71.0.3578.98 Safari/537.36"}


In [4]:
# Your code
response = requests.get(url, headers=headers)
results = response.json()



#### Collect all the objects from selected filters. Total number of pages can be found in the same json. Use *sku* column as index.

Your output should be a Pandas DataFrame of goods. Each row should contain only text or numbers, having family_articles, flags, media and sizes remaining lists (they are exceptions).

In [5]:
# Get the total number of pages
page_count = results["pagination"]["page_count"]
# Your code
flattened_data = json_normalize(results)
flattened_data

,articles,articlesToShow,breadcrumbs,carouselTeaser,categoryTree,collection,contentPositions.entry-point-teasers,contentPositions.in-cat-carousel,contentPositions.in-cat-carousel-fullwidth,contentPositions.in-cat-carousel-mobile,...,total_article_count,total_count,upperInCatTeaser,variants.fullWidthCatalog,variants.hideCategories,variants.mobileLightFilters,variants.myBrandsFilter,variants.outwardTeaserCard,variants.premiumCatalog,wishlist
0,"[{'sku': 'ES124A08B-K11', 'name': 'PANTS - Jea...",84,"[{'items': [{'label': 'Enfant', 'url_key': 'en...",None,"[{'label': 'Promotions', 'id': '9574', 'url_ke...",None,"[7, 14, 20, 26]",9,8,6,...,19568,19566,None,False,False,False,True,False,False,None


In [6]:
flattened_data_art = json_normalize(flattened_data.articles[0])


In [7]:
df = pd.DataFrame()

In [8]:

for page in range(page_count):
    url="https://www.zalando.fr/api/catalog/articles?categories=promo-enfant&limit=84&offset="+str(page*84)+"&sort=sale"
    response = requests.get(url, headers=headers)
    results = response.json()
    flattened_data = json_normalize(results)
    flattened_data_art = json_normalize(flattened_data.articles[0])
    art = flattened_data_art.set_index("sku")
    df = df.append(art)
    
    
df.head()

    

/home/gus/.local/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


,amount,brand_name,family_articles,flags,is_premium,media,name,price.has_different_original_prices,price.has_different_prices,price.has_different_promotional_prices,price.has_discount_on_selected_sizes_only,price.original,price.promotional,product_group,sizes,tracking_information.impression_beacon,tracking_information.metrigo_impression_urls,tracking_information.source,url_key
sku,,,,,,,,,,,,,,,,,,,
N1243D11U-Q11,NaN,Nike Performance,"[{'sku': 'N1243D11U-Q11', 'url_key': 'nike-per...","[{'key': 'sponsored', 'value': 'Sponsorisé', '...",False,[{'path': 'N1/24/3D/11/UQ/11/N1243D11U-Q11@12....,DRY - T-shirt imprimé - black/white,False,True,True,False,"24,95 €","19,95 €",clothing,"[6-8a, 8-10a, 10-12a, 12-13a, 13-15a]",https://ccp-et.metrigo.zalan.do/event/sbv?z=fc...,[https://ccp-et.metrigo.zalan.do/event/sbv?z=f...,ccp,nike-performance-cr7-dry-t-shirt-de-sport-blac...
NI114D09A-Q13,NaN,Nike Sportswear,"[{'sku': 'NI114D09A-Q13', 'url_key': 'nike-spo...","[{'key': 'sponsored', 'value': 'Sponsorisé', '...",False,[{'path': 'NI/11/4D/09/AQ/13/NI114D09A-Q13@6.j...,AIR MAX 720 - Baskets basses - grey/orange,False,True,True,False,"149,95 €","90,00 €",shoe,"[35.5, 36, 36.5, 37.5, 38, 38.5, 39]",https://ccp-et.metrigo.zalan.do/event/sbv?z=fc...,[https://ccp-et.metrigo.zalan.do/event/sbv?z=f...,ccp,nike-sportswear-air-max-720-baskets-basses-ni1...
N1243L00A-Q11,NaN,Nike Performance,"[{'sku': 'N1243L00A-Q11', 'url_key': 'nike-per...","[{'key': 'sponsored', 'value': 'Sponsorisé', '...",False,[{'path': 'N1/24/3L/00/AQ/11/N1243L00A-Q11@11....,SKIRT AIR - Jupe de sport - black/white,False,False,False,False,"34,95 €","24,46 €",clothing,"[8-10a, 10-12a, 12-13a, 13-15a]",https://ccp-et.metrigo.zalan.do/event/sbv?z=fc...,[https://ccp-et.metrigo.zalan.do/event/sbv?z=f...,ccp,nike-performance-skirt-air-jupe-de-sport-black...
CO423G015-G11,NaN,Converse,"[{'sku': 'CO423G015-G11', 'url_key': 'converse...","[{'key': 'discountRate', 'value': '-70%', 'tra...",False,[{'path': 'CO/42/3G/01/5G/11/CO423G015-G11@8.j...,STAR TRIM TEE - T-shirt imprimé - enamel red,False,False,False,False,"28,95 €","8,70 €",clothing,"[8-10a, 10-12a, 12-13a]",NaN,NaN,NaN,converse-star-trim-tee-t-shirt-imprime-co423g0...
N1243E0J0-Q11,NaN,Nike Performance,"[{'sku': 'N1243E0J0-Q11', 'url_key': 'nike-per...","[{'key': 'discountRate', 'value': '-70%', 'tra...",False,[{'path': 'N1/24/3E/0J/0Q/11/N1243E0J0-Q11@4.j...,AIR PANT - Pantalon de survêtement - black,False,False,False,False,"44,95 €","13,49 €",clothing,"[6-8a, 8-10a, 10-12a, 12-13a, 13-15a]",NaN,NaN,NaN,nike-performance-air-pant-pantalon-de-survetem...


#### Display the trending brand in DataFrame

In [16]:
df_b = pd.DataFrame(df["brand_name"].value_counts())
df_b.head()

,brand_name
Friboo,894
OVS,778
Name it,677
Benetton,583
Esprit,483


#### Display the brand with maximal total discount (sum of discounts on all goods)

In [18]:
#Our data is still text. Convert prices into numbers.
# your code

df["price.original"] = df["price.original"].str.extract('(\d*,\d*)')
df["price.promotional"] = df["price.promotional"].str.extract('(\d*,\d*)')


AttributeError: Can only use .str accessor with string values, which use np.object_ dtype in pandas

In [19]:
df["price.original"] = df["price.original"].str.replace(",",".").astype(float)
df["price.promotional"] = df["price.promotional"].str.replace(",",".").astype(float)

AttributeError: Can only use .str accessor with string values, which use np.object_ dtype in pandas

In [20]:
df["discount"] = df["price.original"] - df["price.promotional"]

In [26]:
disc = df.groupby("brand_name")["discount"].sum()
disc.sort_values(ascending=False)

brand_name
Friboo                     10231.70
Polo Ralph Lauren           8024.58
Naturino                    7513.85
Tommy Hilfiger              5498.62
Benetton                    5380.67
THE ANIMALS OBSERVATORY     5295.29
Name it                     5174.32
Nike Performance            5159.66
Esprit                      4982.55
adidas Performance          4957.77
OVS                         4803.51
Puma                        4768.68
Molo                        4571.00
s.Oliver                    4322.67
Nike Sportswear             4241.46
Abercrombie & Fitch         3877.21
adidas Originals            3847.22
Calvin Klein Jeans          3498.86
River Island                3358.57
Pinko Up                    3251.35
Vingino                     3144.71
Petrol Industries           3116.47
Pepe Jeans                  3012.05
Geox                        2889.98
Falcotto                    2848.99
Patrizia Pepe               2733.93
GAP                         2706.12
Emporio Armani   

#### Display the brands without discount at all

In [50]:
# your code
discounts = df[df["discount"] == 0]
disc.loc[disc == 0] # Ther's no brand without discounts.




Series([], Name: discount, dtype: float64)